<a href="https://colab.research.google.com/github/andromeda123248/Dip/blob/main/Transformer_dip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
! pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 20.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=7087b9508fb8d2d3eb05512671c6f58cca4f7ccad34bb49513cf49e50d7650a7
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [3]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pymorphy2
import re

In [4]:
import nltk
nltk.download('punkt')

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
! pip install faiss-gpu ##Installing GPU version of faiss



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 5.5 MB/s eta 0:00:00


In [6]:
! pip install json

ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


In [ ]:
! pip install sentence_transformers ## For textual similarity, using pretrained models

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)


In [ ]:
import faiss
from sentence_transformers import SentenceTransformer, util

In [ ]:
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

### Load the movies data

In [ ]:
# Укажите путь к вашему файлу
json_file_path = 'ex_tr'

# Прочтите JSON файл и преобразуйте его в DataFrame
df = pd.read_json(json_file_path)

# Распаковываем столбец 'docs' в отдельные столбцы 'name', 'description' и 'genres'
df[['name', 'description', 'genres']] = df['docs'].apply(
        lambda x: pd.Series([x['name'], x['description'], ', '.join([genre['name'] for genre in x['genres']])]))

# Удаляем столбец 'docs', так как мы уже распаковали его содержимое
df = df.drop(columns=['docs'])

# Просмотрите первые несколько строк DataFrame
print(df.head())

In [ ]:
df

In [ ]:
df['id']=df.index

In [ ]:
df

In [ ]:
sentences = df['description'].tolist()
print("Number of Sentences in Movie Description ",len(sentences))

In [ ]:
embeddings = model.encode(sentences)
faiss.normalize_L2(embeddings) ## Normalising the Embeddings

In [ ]:
print("Shape of the EMbeddings is ",embeddings.shape)

In [ ]:
dim=768
ncentroids=50 ## This is a hyperparameter, and indicates number of clusters to be split into
m=16 ## This is also a hyper parameter
quantiser = faiss.IndexFlatL2(dim)
index = faiss.IndexIVFPQ (quantiser, dim,ncentroids, m , 8)
index.train(embeddings) ## This step, will do the clustering and create the clusters
print(index.is_trained)
faiss.write_index(index, "trained.index")

In [ ]:
### We have to add the embeddings to the Trained Index.
ids = df['id'].tolist()
ids = np.array(ids)
index.add_with_ids(embeddings,ids)
print(index.ntotal)

In [ ]:
faiss.write_index(index,"block.index")

In [ ]:
def searchFAISSIndex(data,id_col_name,query,index,nprobe,model,topk=20):
    ## Convert the query into embeddings
    query_embedding=model.encode([query])[0]
    dim=query_embedding.shape[0]
    query_embedding=query_embedding.reshape(1,dim)
    faiss.normalize_L2(query_embedding)


    index.nprobe=nprobe

    D,I=index.search(query_embedding,topk)
    ids=[i for i in I][0]
    L2_score=[d for d in D][0]
    inner_product=[calculateInnerProduct(l2) for l2 in L2_score]
    search_result=pd.DataFrame()
    search_result[id_col_name]=ids
    search_result['cosine_sim']=inner_product
    search_result['L2_score']=L2_score
    dat=data[data[id_col_name].isin(ids)]
    dat=pd.merge(dat,search_result,on=id_col_name)
    dat=dat.sort_values('cosine_sim',ascending=False)
    return dat

In [ ]:
def calculateInnerProduct(L2_score):
    return (2-math.pow(L2_score,2))/2

In [ ]:
query="миллионер особняк"
search_result=searchFAISSIndex(df,"id",query,index,nprobe=10,model=model,topk=20)
search_result=search_result[['id','description','name','cosine_sim','L2_score']]

In [ ]:
search_result